# Constants & Libs

In [1]:
from pathlib import Path
import os
from tqdm import tqdm

In [2]:
os.getcwd()

'D:\\PythonProjects\\AML_test_task\\exploration\\notebooks'

In [3]:
PATH_TO_PROJECT = Path(os.getcwd()).parent.parent

In [4]:
os.chdir(PATH_TO_PROJECT)

In [5]:
PATH_TO_PROJECT

WindowsPath('D:/PythonProjects/AML_test_task')

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import re

from exploration.Modules.CheckerClass import Checker
from exploration.Modules.VisualisationClass import Vizualization

from feature_engine.encoding import MeanEncoder

from sklearn.preprocessing import StandardScaler

from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from feature_engine.imputation import ArbitraryNumberImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import calibration_curve
from sklearn.calibration import CalibratedClassifierCV
from matplotlib import pyplot
from sklearn import cluster
from sklearn.preprocessing import MinMaxScaler

In [8]:
from boruta import BorutaPy

In [9]:
from sklearn.ensemble import IsolationForest

In [10]:
from feature_engine.imputation import AddMissingIndicator

In [11]:
from sklearn.model_selection import GridSearchCV
import joblib

In [12]:
# Imports from LightAutoML package
# from lightautoml.automl.presets.tabular_presets import TabularAutoML
# from lightautoml.tasks import Task

In [13]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,VotingClassifier

In [14]:
import lightgbm as lgb
import xgboost as xgb

In [15]:
PATH_TO_TRAIN_DATA = PATH_TO_PROJECT.joinpath('data/raw_data/train.csv')
PATH_TO_TEST_DATA = PATH_TO_PROJECT.joinpath('data/raw_data/test.csv')

In [16]:
PATH_TO_ARTEFACTS = PATH_TO_PROJECT.joinpath('exploration/artefacts')

In [17]:
PATH_TO_PREPARED_DATA = PATH_TO_PROJECT.joinpath('data/prepared_data')

# Read file

In [ ]:
train_data = pd.read_csv(PATH_TO_TRAIN_DATA)

In [ ]:
test_data = pd.read_csv(PATH_TO_TEST_DATA)

# EDA Train data

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
# совпадает ли Id с индексом
sum(test_data['Id'].values != test_data.index.values)

In [ ]:
# удаление столбца Id
test_data.drop(columns='Id', inplace=True)

## watch types, NaNs and etc.

In [ ]:
train_data.info()

In [ ]:
test_data.info()

### types

In [ ]:
for col in train_data.columns:
    print(col)
    print(train_data[col].map(type).unique(), '\n')

In [ ]:
for col in test_data.columns:
    print(col)
    print(test_data[col].map(type).unique(), '\n')

fact_dt только строковый тип, необходимо получить числа

столбцы в train и test в одинаковых форматах

### get days from fact_dt

In [ ]:
train_data

выделяем количество дней из строк в fact_dt

In [ ]:
def get_days(data: pd.DataFrame, col: str):
    day_numbers = []
    for amount_days in data[col]:
        days = int(re.search(r'[0-9]+', amount_days)[0])
        day_numbers.append(days)
    data[col] = day_numbers
    return data

In [ ]:
train_data = get_days(train_data, 'fact_dt')
test_data = get_days(test_data, 'fact_dt')

### describe features

In [ ]:
train_data.describe()

In [ ]:
test_data.describe()

макс. значения у entity_0015_ct, entity_0008_ct, entity_0001_ct  разлицаются в несколько раз, возможно выбросы

In [ ]:
train_data.groupby('target').agg(['mean', 'min', 'max'])

cтолбец entity_0016_rt полностью разделяет таргет

столбцы с rt видимо должны быть >= 0, но entity_0016_rt < 0, cкорее всего выброс

столбец entity_0007_ct есть только для таргета 1

In [ ]:
# сделаем идентификатор, чтобы посмотреть различия в значениях train и test
train_data['train_or_test'] = 'train'
test_data['train_or_test'] = 'test'

In [ ]:
pd.concat([train_data, test_data]).groupby('train_or_test').agg(['mean', 'min', 'max'])

cтолбец entity_0016_rt имеет абсолютно разные значение на train и test
возможно, стоит его вообще убрать или как-то преобразовать

часть фичей из эмбедингов могут полностью лежать в области < 0 (например emb_0020) на train, но при этом могут иметь значения > 0 на test, возможно выбросы

In [ ]:
# Vizualization(train_data).numTargetHist(target='target', 
#                                         save_folder=f'{PATH_TO_ARTEFACTS}/train_features_target.png')

In [ ]:
# Vizualization(pd.concat([train_data, test_data])).numTargetHist(
#     target='train_or_test', 
#     save_folder=f'{PATH_TO_ARTEFACTS}/train_test_features.png'
# )

небольшая доля user_id повторяется в train и test

небольшая доля fact_dt повторяется в train и test, но в целом в test значения больше (возможно там более новые значения)

In [ ]:
for col in train_data.columns:
    print(col)
    print(train_data[col].value_counts().iloc[:15], '\n')

таргет несбалансирован

In [ ]:
for col in test_data.columns:
    print(col)
    print(test_data[col].value_counts().iloc[:10], '\n')

категориальная переменная одна, категории одинаковые для train и test

#### change values

возможно допущена ошибка в данных entity_0016_rt, необходимо преобразовать, иначе невозможно использовать

In [ ]:
train_data['entity_0016_rt'] = np.where(train_data['entity_0016_rt'] < 0.5, 
                                        train_data['entity_0016_rt']+1,
                                        train_data['entity_0016_rt'])

In [ ]:
test_data['entity_0016_rt'] = np.where(test_data['entity_0016_rt'] < 0.5, 
                                       test_data['entity_0016_rt']+1,
                                       test_data['entity_0016_rt'])

преобразовываем категориальные переменные

In [ ]:
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)

In [ ]:
train_data.shape, test_data.shape

#### drop features

In [ ]:
# убираем лишнюю колонку
train_data.drop(columns=['train_or_test_train'], inplace=True)
test_data.drop(columns=['train_or_test_test'], inplace=True)

### check duplicates

In [ ]:
# проверка на повторяющиеся строки
len(train_data[train_data.duplicated()]), len(test_data[test_data.duplicated()])

In [ ]:
# проверка на повторяющихся юзеров
train_data[train_data.duplicated(subset='user_id', keep=False)]

In [ ]:
# величина таргета для повторяющихся юзеров
train_data[train_data.duplicated(subset='user_id', keep=False)].mean()['target']

In [ ]:
test_data[test_data.duplicated(subset='user_id', keep=False)]

#### generate features

In [ ]:
train_data[train_data.duplicated(subset='user_id', keep=False)].mean()

повторяющихся юзеров можно использовать как допю фичу

In [ ]:
# определение списка повторяющихся юзеров
duplicated_users_train = train_data[train_data.duplicated(subset='user_id', keep=False)]['user_id'].unique()
duplicated_users_test = test_data[test_data.duplicated(subset='user_id', keep=False)]['user_id'].unique()

In [ ]:
# создание доп фичи repeat_user
train_data['repeat_user'] = np.where(train_data['user_id'].isin(duplicated_users_train), 
                                    1, 
                                    0)

test_data['repeat_user'] = np.where(test_data['user_id'].isin(duplicated_users_test), 
                                    1, 
                                    0)

In [ ]:
# определение уникальных пользователей в train и test
unique_id_train = set(train_data['user_id'].unique())
unique_id_test = set(test_data['user_id'].unique())                      

In [ ]:
# кол-во одинаковых пользователей в train и test
len(unique_id_train & unique_id_test)

In [ ]:
train_data[train_data['user_id'].isin(unique_id_test)]

#### drop features

In [ ]:
# удаление признака user_id
train_data.drop(columns='user_id', inplace=True)
test_data.drop(columns='user_id', inplace=True)

### check NaNs

In [ ]:
# определение процента пропусков в столбцах train
Checker(train_data).check_missing()

In [ ]:
# определение процента пропусков в столбцах test
Checker(test_data).check_missing()

видно, что пропуски в одних и тех же колонках, при чем у одних и тех же колонках процент пропусков одинаковый, в качестве доп. признаков можно использовать индикаторы пропусков

#### generate new features

In [ ]:
ami = AddMissingIndicator()

In [ ]:
train_data = ami.fit_transform(train_data)

In [ ]:
test_data = ami.fit_transform(test_data)

In [ ]:
train_data

проверим, на одних ли и тех же местах находятся пропуски, если да, то такие фичи одинаковы, часть из них надо удалить

In [ ]:
(train_data['entity_0015_ct_na'] == train_data['entity_0008_ct_na']).sum()

In [ ]:
(train_data['entity_0006_rt_na'] == train_data['entity_0012_ct_na']).sum()

#### drop features

In [ ]:
train_data.drop(columns=['entity_0008_ct_na', 'entity_0012_ct_na', 'entity_0014_rt_na'], inplace=True)
test_data.drop(columns=['entity_0008_ct_na', 'entity_0012_ct_na', 'entity_0014_rt_na'], inplace=True)

### correlations

оценим степень уровень корреляции

In [ ]:
corr_data = Checker(train_data).featureCorrelation(target='target', corr_edge=0.7)

In [ ]:
corr_data['negativeCorr']

In [ ]:
corr_data['positiveCorr']

### find outliers

In [ ]:
train_data.columns

In [ ]:
cols_to_drop=['target', 'entity_0017_cd_a', 'entity_0017_cd_b', 'entity_0017_cd_c',
       'entity_0017_cd_d', 'repeat_user', 'entity_0006_rt_na',
       'entity_0007_ct_na', 'entity_0009_rt_na', 'entity_0010_rt_na',
       'entity_0015_ct_na']

In [ ]:
features_train_for_outliers = train_data.drop(columns=cols_to_drop).copy()
features_test_for_outliers = test_data.drop(columns=cols_to_drop[1:]).copy()

In [ ]:
features_train_for_outliers.fillna(-9999, inplace=True)
features_test_for_outliers.fillna(-9999, inplace=True)

In [ ]:
features_train_for_outliers

In [ ]:
features_test_for_outliers

In [ ]:
outliers_finder=IsolationForest(random_state=42, verbose=0)

In [ ]:
for col in tqdm(features_train_for_outliers.columns):
    outliers_finder.fit(features_train_for_outliers[[col]])
    anomalies_detected_train = outliers_finder.predict(features_train_for_outliers[[col]])
    anomalies_detected_test = outliers_finder.predict(features_test_for_outliers[[col]])
    train_data[f'{col}_outlier'] = anomalies_detected_train
    test_data[f'{col}_outlier'] = anomalies_detected_test

In [ ]:
train_data

In [ ]:
test_data

### generate math features

In [ ]:
embeding_columns = [col for col in train_data.columns if 'emb' in col and 'outlier' not in col]
embeding_columns

In [ ]:
count_columns = [col for col in train_data.columns if 'ct' in col]
count_columns

In [ ]:
ratio_columns = [col for col in train_data.columns if 'rt' in col]
ratio_columns

In [ ]:
def math_embeding_columns(data, columns):
    embedings_features = data[columns].copy()
    embedings_features['mul_emb'] = embedings_features.prod(axis=1)
    embedings_features['sum_emb'] = embedings_features.sum(axis=1)
    embedings_features['sum_abs_emb'] = abs(embedings_features).sum(axis=1)
    embedings_features.drop(columns=columns, inplace=True)
    return embedings_features

In [ ]:
def math_count_columns(data, columns):
    count_features = data[columns].copy()
    count_features['mul_ct'] = count_features.fillna(1).prod(axis=1)
    count_features['sum_ct'] = count_features.fillna(0).sum(axis=1)
    count_features.drop(columns=columns, inplace=True)
    return count_features

In [ ]:
def math_ratio_columns(data, columns):
    ratio_features = data[columns].copy()
    ratio_features['mul_rt'] = ratio_features.fillna(1).prod(axis=1)
    ratio_features['sum_rt'] = ratio_features.fillna(0).sum(axis=1)
    ratio_features.drop(columns=columns, inplace=True)
    return ratio_features

In [ ]:
train_data[ratio_columns]

In [ ]:
train_data_ratio = math_ratio_columns(train_data, ratio_columns)
test_data_ratio = math_ratio_columns(test_data, ratio_columns)

In [ ]:
train_data_math = math_count_columns(train_data, count_columns)
test_data_math = math_count_columns(test_data, count_columns)

In [ ]:
train_data_embeding = math_embeding_columns(train_data, embeding_columns)
test_data_embeding = math_embeding_columns(test_data, embeding_columns)

In [ ]:
train_data = pd.concat([train_data, train_data_ratio, train_data_math, train_data_embeding], axis=1)
test_data = pd.concat([test_data, test_data_ratio, test_data_math, test_data_embeding], axis=1)

In [ ]:
test_data

### feature selection

In [ ]:
features_train_for_feature_selection = train_data.drop(columns='target').copy()
target_train_for_feature_selection = train_data['target']

In [ ]:
features_train_for_feature_selection

In [ ]:
features_train_for_feature_selection.fillna(-9999, inplace=True)

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=10)

In [ ]:
feature_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

In [ ]:
feature_selector.fit(features_train_for_feature_selection.values, 
                     target_train_for_feature_selection.values)

In [ ]:
feature_selector.support_

In [ ]:
len(feature_selector.support_)

In [ ]:
need_cols = []
for col, status in zip(features_train_for_feature_selection.columns, feature_selector.support_):
    if status==True:
        need_cols.append(col)

In [ ]:
# check ranking of features
feature_selector.ranking_

In [ ]:
need_cols

### make features_train, features_test, target_train

In [ ]:
features_train = train_data[need_cols]
target_train = train_data['target']

In [ ]:
features_test = test_data[need_cols]

### generate features (clusters)

In [ ]:
pipeline_stages = []

pipeline_stages.append(("imputer", ArbitraryNumberImputer(arbitrary_number=-9999)))
pipeline_stages.append(("scaler", MinMaxScaler()))
pipeline_stages.append(("cluster", cluster.KMeans(n_clusters = 7)))

# Собираем все шаги в пайплайн
pipeline = Pipeline(pipeline_stages)

In [ ]:
pipeline.fit(features_train)

In [ ]:
features_train['clusters'] = pipeline.predict(features_train).astype(str)
features_test['clusters'] = pipeline.predict(features_test).astype(str)

In [ ]:
features_train

In [ ]:
features_test

In [ ]:
features_train.to_pickle(f'{PATH_TO_PREPARED_DATA}/features_train.pickle')
features_test.to_pickle(f'{PATH_TO_PREPARED_DATA}/features_test.pickle')
target_train.to_pickle(f'{PATH_TO_PREPARED_DATA}/target_train.pickle')

# Models

In [36]:
best_model = joblib.load(f'{PATH_TO_ARTEFACTS}/model.pickle')

In [37]:
best_model.get_params()

{'memory': None,
 'steps': [('imputer', None),
  ('scaler', None),
  ('model', <catboost.core.CatBoostClassifier at 0x20d3411a948>)],
 'verbose': False,
 'imputer': None,
 'scaler': None,
 'model': <catboost.core.CatBoostClassifier at 0x20d3411a948>,
 'model__l2_leaf_reg': 10,
 'model__verbose': 0,
 'model__max_depth': 10,
 'model__n_estimators': 1000,
 'model__random_state': 42,
 'model__min_data_in_leaf': 1}

## read prepared files

In [18]:
features_train = pd.read_pickle(f'{PATH_TO_PREPARED_DATA}/features_train.pickle')
features_test = pd.read_pickle(f'{PATH_TO_PREPARED_DATA}/features_test.pickle')
target_train = pd.read_pickle(f'{PATH_TO_PREPARED_DATA}/target_train.pickle')

In [19]:
features_train

,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0008_ct,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0017_cd_a,entity_0017_cd_d,entity_0010_rt_na,entity_0015_ct_na,fact_dt_outlier,emb_0045_outlier,entity_0006_rt_outlier,entity_0008_ct_outlier,entity_0010_rt_outlier,entity_0012_ct_outlier,entity_0013_ct_outlier,entity_0015_ct_outlier,mul_rt,sum_rt,mul_ct,sum_ct,sum_emb,sum_abs_emb
0,711,-1.130920,1.895028,-1.736251,0.872448,-0.854317,0.545024,1.202575,-0.072018,-1.165497,1.651011,-1.892026,1.972686,-2.306150,-0.285928,-2.420561,0.831787,1.104704,-1.665488,0.751251,0.845312,-0.542118,2.076014,1.700184,0.182223,2.269419,-0.768279,-0.777146,0.387730,-0.143139,0.634687,-0.858275,0.777643,-0.325165,0.527793,0.294793,-0.937137,0.134326,-0.874896,-0.969246,-0.625632,-1.427790,2.489812,-1.845182,0.308496,0.466395,0.023107,-0.570620,-1.263162,2.243511,-0.771934,-1.779594,1.260181,2.347770,0.153677,0.483990,-3.030639,-1.347744,1.124366,-1.342232,1.381671,-0.332619,-0.967959,0.195158,-0.251763,3,4,3,0,1,0.993584,NaN,1.000000,3,446.0,207,1.166190,NaN,0,1,0,1,-1,1,1,1,1,1,1,1,1.168426,5.336587,0.000000e+00,1.378000e+03,-2.146653,70.562856
1,218,-1.777758,2.195476,-1.177928,0.997776,-0.966194,0.702989,0.975820,0.703755,-0.958513,0.480120,-1.259643,1.984836,-2.209633,0.655391,-2.626339,0.814797,0.706640,-1.508253,0.892528,1.047994,-1.287013,2.281535,1.653299,0.457043,2.041678,-1.223240,-1.323192,-0.318703,0.516459,0.097562,-1.596383,0.115891,-0.660581,0.566672,0.061095,-0.824538,0.678029,-0.629933,-0.288293,-0.191039,-1.304114,1.895253,-0.748206,1.133617,0.867598,0.552327,-0.532176,-0.611312,2.055071,-0.842618,-1.081904,1.463756,1.909271,-0.597187,0.573740,-3.777002,-1.020027,0.414380,-1.603858,1.519238,-0.792133,-0.571999,-0.327029,-0.506496,2,2,2,2,2,0.985236,NaN,0.996865,3,330.0,200,1.753914,NaN,0,1,0,1,1,1,1,1,-1,1,1,1,1.681920,7.394348,1.381248e+09,1.381249e+09,-2.131585,70.286472
2,462,-0.800151,2.204143,-0.514011,0.961015,-0.116245,0.218591,1.152742,0.955213,-1.075719,2.451778,-3.058330,2.097131,-2.285532,0.062348,-2.129199,0.829396,0.642969,-1.849109,-0.553088,0.575978,-0.848553,1.657650,1.695016,0.018969,2.155971,-0.469931,-0.967494,-0.895899,-0.364366,0.944774,-0.673376,0.791378,-0.214279,0.588130,0.450273,-0.892531,0.754762,-0.814439,-0.444689,-0.521155,-0.792310,2.015573,-1.363314,0.924737,0.155714,-0.670974,-0.793958,-0.905191,2.167285,-1.523553,-0.857246,1.325879,1.684667,0.385727,0.096711,-2.704222,-1.160303,0.934533,-1.503963,-0.016609,-0.465335,-1.034321,0.653358,-0.727603,4,0,4,0,2,0.987922,NaN,0.781602,2,191.0,82,1.548657,NaN,0,0,0,1,1,-1,1,1,-1,1,1,1,1.183690,5.491732,0.000000e+00,7.470000e+02,-2.454584,68.013993
3,13,-1.543821,2.044538,-1.288806,0.958519,-0.485421,0.534662,0.498838,0.913363,-1.150698,0.682353,-2.342354,1.619433,-2.008181,0.317489,-2.045199,0.819717,0.359040,-1.013714,0.515654,0.962981,-0.895936,2.444055,1.650715,0.488509,2.050253,-0.513660,-1.617719,-0.198799,0.332519,0.498607,-1.393829,0.163533,-0.217686,0.988170,-0.014833,-1.646648,1.749996,0.100091,-0.908938,-1.034925,-1.161342,2.306280,-1.087395,0.695601,1.123252,0.504989,-0.053581,-1.101684,2.047417,-0.872976,-1.436043,1.493698,1.900455,-0.137229,-0.173302,-3.522463,-1.296566,0.875170,-1.407083,0.554178,-0.708667,-1.267630,0.610148,-0.363370,4,4,4,0,2,0.990692,NaN,0.000000,4,73

In [20]:
features_test

,fact_dt,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,emb_0009,emb_0010,emb_0011,emb_0012,emb_0013,emb_0014,emb_0015,emb_0016,emb_0017,emb_0018,emb_0019,emb_0020,emb_0021,emb_0022,emb_0023,emb_0024,emb_0025,emb_0026,emb_0027,emb_0028,emb_0029,emb_0030,emb_0031,emb_0032,emb_0033,emb_0034,emb_0035,emb_0036,emb_0037,emb_0038,emb_0039,emb_0040,emb_0041,emb_0042,emb_0043,emb_0044,emb_0045,emb_0046,emb_0047,emb_0048,emb_0049,emb_0050,emb_0051,emb_0052,emb_0053,emb_0054,emb_0055,emb_0056,emb_0057,emb_0058,emb_0059,emb_0060,emb_0061,emb_0062,emb_0063,entity_0001_ct,entity_0002_ct,entity_0003_ct,entity_0004_ct,entity_0005_ct,entity_0006_rt,entity_0008_ct,entity_0010_rt,entity_0011_ct,entity_0012_ct,entity_0013_ct,entity_0014_rt,entity_0015_ct,entity_0017_cd_a,entity_0017_cd_d,entity_0010_rt_na,entity_0015_ct_na,fact_dt_outlier,emb_0045_outlier,entity_0006_rt_outlier,entity_0008_ct_outlier,entity_0010_rt_outlier,entity_0012_ct_outlier,entity_0013_ct_outlier,entity_0015_ct_outlier,mul_rt,sum_rt,mul_ct,sum_ct,sum_emb,sum_abs_emb
0,552,-1.786346,2.074665,-1.504628,1.143300,-0.979666,0.750931,1.091962,0.644968,-0.810258,1.023059,-1.576939,1.783760,-1.997192,0.375575,-2.233522,0.824557,0.603287,-1.959785,0.645056,1.101540,-1.009328,2.402104,1.685323,0.386604,2.042387,-0.910862,-1.246986,-0.331123,0.027410,0.443654,-1.291464,0.301381,-0.655716,-0.372735,0.160851,-0.503871,1.123703,-0.950976,-0.205005,-0.654555,-1.551749,1.944104,-1.391381,0.745322,0.609470,0.234558,0.099144,-1.020924,2.415789,-0.856164,-1.336662,1.386519,2.437416,-0.077037,0.320741,-3.254244,-1.314506,0.722575,-1.563255,1.282448,-0.733094,-0.804640,0.264208,-0.284033,2,4,2,1,2,0.974990,NaN,0.997459,3,339.0,173,2.300125,NaN,0,1,0,1,1,1,1,1,-1,1,1,1,2.228614,8.497483,3.107822e+09,3.107823e+09,-2.070279,70.337293
1,152,-1.555227,1.703838,-1.682477,1.143936,-1.363388,0.921123,1.209163,0.331891,-0.998299,1.015320,-1.242066,1.766697,-2.308836,0.442604,-2.749990,0.825489,0.596908,-1.176754,0.788825,0.962762,-1.042709,2.203321,1.695730,0.675732,2.172944,-1.027670,-1.245881,-0.080473,0.086483,0.430647,-1.400723,0.373409,-0.274544,0.677400,0.289406,-0.300185,0.725815,-0.903123,0.062829,-0.462003,-1.259109,2.258573,-1.285355,0.658545,0.672804,0.299276,-0.514287,-0.872554,2.333222,-0.890981,-1.186845,1.406470,2.004209,-0.169116,0.444842,-3.829849,-1.207255,0.239036,-1.438659,1.806309,-0.535842,-0.715147,-0.243989,-0.578661,2,1,2,1,0,0.987894,NaN,NaN,1,98.0,54,1.598273,NaN,1,0,1,1,-1,1,1,1,-1,1,1,1,1.563576,6.140022,0.000000e+00,3.110000e+02,-1.316440,69.083997
2,714,-1.350453,1.907620,-1.563653,1.271580,-0.902961,0.582773,1.127565,0.447938,-1.022802,0.711931,-1.636709,1.885665,-2.092236,0.163758,-2.515830,0.828522,0.918868,-1.392947,1.453549,1.095837,-0.908404,2.470779,1.678719,-0.205788,2.162640,-1.063902,-1.219050,0.076042,0.341267,0.533246,-1.241235,0.506563,-0.416503,0.272600,0.325620,-0.808660,0.350219,-0.941883,-0.332984,-0.770360,-1.138200,2.354686,-1.409007,0.793536,0.674541,0.447000,-0.143163,-1.145194,1.947757,-0.832913,-1.443164,1.409840,2.089422,-0.462843,0.211531,-3.950086,-1.224400,0.911566,-1.581317,1.515879,-0.571897,-0.961354,-0.128131,-0.358940,2,2,2,1,1,0.991317,NaN,1.000000,2,235.0,109,1.368001,NaN,0,1,0,1,-1,1,1,1,1,1,1,1,1.340159,6.687705,2.926258e+08,2.926268e+08,-2.267905,71.473937
3,5,-1.757556,2.429529,-1.438695,0.913385,0.016773,0.629686,1.096519,0.901125,-0.639822,0.610012,-2.448452,2.254705,-1.977030,0.244674,-2.132576,0.832347,0.174011,-1.245904,0.680392,1.120218,-0.973336,1.885519,1.641319,-0.427695,2.185217,-0.738151,-1.244538,-0.098398,0.586035,0.734197,-1.173107,0.733088,0.058661,0.360945,0.124782,-0.381303,0.442166,-1.142064,-0.315166,-0.944775,-0.986942,2.199443,-1.143928,1.136280,0.724500,0.491751,0.133391,-1.089635,1.786158,-1.264123,-1.255365,1.565839,1.854973,-0.334534,-0.233716,-3.841123,-1.307208,1.019935,-1.614235,0.013505,-0.506440,-1.273173,0.367237,-0.853831,1,0,1,0,0,0.980202,NaN,NaN,1,92.0,19,2.023099,NaN

## baseline model

### catboost

In [41]:
pipeline_stages = []

# pipeline_stages.append(("imputer", ArbitraryNumberImputer(arbitrary_number=-9999)))

# pipeline_stages.append(("encoder", CountFrequencyEncoder(encoding_method='count')))
# pipeline_stages.append(("encoder", MeanEncoder()))

# pipeline_stages.append(("imputer", SimpleImputer(strategy='median')))

# pipeline_stages.append(("scaler", MinMaxScaler()))
# pipeline_stages.append(("cluster", cluster.KMeans(n_clusters = 5)))
# pipeline_stages.append(("poly_features", PolynomialFeatures(degree=2, interaction_only=True)))

pipeline_stages.append(("model", CatBoostClassifier(verbose=0, n_estimators=1000)))
# pipeline_stages.append(("model", LogisticRegression()))
# pipeline_stages.append(("model", IsolationForest()))


# Собираем все шаги в пайплайн
pipeline = Pipeline(pipeline_stages)

In [42]:
tscv = StratifiedKFold(n_splits=5)

In [43]:
metric_results = []
for train_idx, valid_idx in tscv.split(features_train, target_train):
    # разбиваем по фолдам
    X_train, X_test = features_train.iloc[train_idx], features_train.iloc[valid_idx]
    y_train, y_test = target_train.iloc[train_idx], target_train.iloc[valid_idx]
    # обучаем модель и считаем метрики
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict_proba(X_test)
    metric_results.append(roc_auc_score(y_test, y_pred[:, 1]))
    print(roc_auc_score(y_test, y_pred[:, 1]))
print(np.mean(metric_results))

0.8084513166493845
0.8037686526496738
0.8102737598461724
0.8195899128450237
0.8702991995870901
0.822476568315469


### lgbm

In [47]:
pipeline_stages = []

# pipeline_stages.append(("imputer", ArbitraryNumberImputer(arbitrary_number=-9999)))

# pipeline_stages.append(("encoder", CountFrequencyEncoder(encoding_method='count')))
# pipeline_stages.append(("encoder", MeanEncoder()))

pipeline_stages.append(("imputer", SimpleImputer(strategy='median')))

# pipeline_stages.append(("scaler", MinMaxScaler()))
# pipeline_stages.append(("cluster", cluster.KMeans(n_clusters = 5)))
# pipeline_stages.append(("poly_features", PolynomialFeatures(degree=2, interaction_only=True)))

pipeline_stages.append(("model", lgb.LGBMClassifier(n_estimators=500)))
# pipeline_stages.append(("model", LogisticRegression()))
# pipeline_stages.append(("model", IsolationForest()))


# Собираем все шаги в пайплайн
pipeline = Pipeline(pipeline_stages)

In [48]:
tscv = StratifiedKFold(n_splits=5)

In [49]:
metric_results = []
for train_idx, valid_idx in tscv.split(features_train, target_train):
    # разбиваем по фолдам
    X_train, X_test = features_train.iloc[train_idx], features_train.iloc[valid_idx]
    y_train, y_test = target_train.iloc[train_idx], target_train.iloc[valid_idx]
    # обучаем модель и считаем метрики
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict_proba(X_test)
    metric_results.append(roc_auc_score(y_test, y_pred[:, 1]))
    print(roc_auc_score(y_test, y_pred[:, 1]))
print(np.mean(metric_results))

0.8057378037661014
0.800606861861904
0.8070553235931814
0.8152197243745387
0.8671547048181483
0.8191548836827748


### voting models

In [ ]:
estimator = [
    ('m1', CatBoostClassifier()),
    ('m2', RandomForestClassifier()),
    ('m3', GradientBoostingClassifier()),
    ('m4', (lgb.LGBMClassifier())),
    ('m5', xgb.XGBClassifier()),
    ('m6', LogisticRegression())
]

In [ ]:
pipeline_stages = []

pipeline_stages.append(("imputer", ArbitraryNumberImputer(arbitrary_number=-9999)))
pipeline_stages.append(("encoder", MeanEncoder()))
pipeline_stages.append(("scaler", MinMaxScaler()))
pipeline_stages.append(('model', VotingClassifier(estimators = estimator, voting='soft')))


# Собираем все шаги в пайплайн
pipeline = Pipeline(pipeline_stages)

In [ ]:
tscv = StratifiedKFold(n_splits=5)

In [ ]:
metric_results = []
for train_idx, valid_idx in tscv.split(features_train, target_train):
    # разбиваем по фолдам
    X_train, X_test = features_train.iloc[train_idx], features_train.iloc[valid_idx]
    y_train, y_test = target_train.iloc[train_idx], target_train.iloc[valid_idx]
    # обучаем модель и считаем метрики
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict_proba(X_test)
    metric_results.append(roc_auc_score(y_test, y_pred[:, 1]))
    print(roc_auc_score(y_test, y_pred[:, 1]))
print(np.mean(metric_results))

## find best model

In [ ]:
pipeline_stages = []

pipeline_stages.append(("imputer", ArbitraryNumberImputer(arbitrary_number=-9999)))
pipeline_stages.append(("scaler", StandardScaler()))
pipeline_stages.append(("model", CatBoostClassifier(verbose=0)))

pipeline = Pipeline(pipeline_stages)

In [ ]:
param_grid = [
    {
        "imputer": [SimpleImputer(strategy='median'), ArbitraryNumberImputer(arbitrary_number=-9999)],
        "model": [CatBoostClassifier(random_state=42, verbose=0)],
        "model__n_estimators": [500, 1000],
        "model__max_depth": [6, 8, 10, 12],
        'model__loss_function': ['Logloss', 'CrossEntropy'],
        "model__l2_leaf_reg": [3, 4, 5], 
        "model__min_data_in_leaf": [1, 3, 5],
        "scaler": [None]
    },
    {
        "imputer": [SimpleImputer(strategy='median'), ArbitraryNumberImputer(arbitrary_number=-9999)],
        "model": [LogisticRegression(random_state=42)],
        "model__C": [0.001, 0.01, 0.1, 1, 10, 100],
        "model__penalty":['none', 'elasticnet', 'l1', 'l2'],
        "model__solver": ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        "scaler": [StandardScaler()]
    },
]

In [ ]:
tscv = StratifiedKFold(n_splits=5)

In [ ]:
grid = GridSearchCV(pipeline, param_grid=param_grid, scoring="roc_auc", cv=tscv, verbose=2)

In [ ]:
grid.fit(features_train, target_train)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_

In [ ]:
grid.cv_results_

In [ ]:
calibrated_clf = CalibratedClassifierCV(base_clf, cv=3)

In [ ]:
# predict probabilities
probs = model.predic_proba(testX)[:,1]
# reliability diagram
fop, mpv = calibration_curve(testy, probs, n_bins=10)
# plot perfectly calibrated
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
pyplot.plot(mpv, fop, marker='.')
pyplot.show()

# Make submission

In [ ]:
# pipeline = grid.best_estimator_

In [ ]:
calibrated_clf = CalibratedClassifierCV(base_clf, cv=3)

In [ ]:
# predict probabilities
probs = model.predic_proba(testX)[:,1]
# reliability diagram
fop, mpv = calibration_curve(testy, probs, n_bins=10)
# plot perfectly calibrated
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
pyplot.plot(mpv, fop, marker='.')
pyplot.show()

# Make submission

In [72]:
# pipeline = grid.best_estimator_

In [93]:
pipeline.fit(features_train, target_train)

Learning rate set to 0.075535
0:	learn: 0.6364582	total: 25.3ms	remaining: 25.3s
1:	learn: 0.5919468	total: 49.3ms	remaining: 24.6s
2:	learn: 0.5566472	total: 68.2ms	remaining: 22.7s
3:	learn: 0.5272320	total: 103ms	remaining: 25.6s
4:	learn: 0.5020430	total: 125ms	remaining: 24.8s
5:	learn: 0.4800299	total: 144ms	remaining: 23.9s
6:	learn: 0.4629460	total: 165ms	remaining: 23.4s
7:	learn: 0.4483619	total: 188ms	remaining: 23.3s
8:	learn: 0.4357522	total: 210ms	remaining: 23.1s
9:	learn: 0.4256047	total: 234ms	remaining: 23.1s
10:	learn: 0.4170784	total: 252ms	remaining: 22.7s
11:	learn: 0.4089711	total: 274ms	remaining: 22.5s
12:	learn: 0.4025174	total: 294ms	remaining: 22.4s
13:	learn: 0.3972532	total: 313ms	remaining: 22.1s
14:	learn: 0.3918933	total: 334ms	remaining: 22s
15:	learn: 0.3881027	total: 352ms	remaining: 21.6s
16:	learn: 0.3838804	total: 372ms	remaining: 21.5s
17:	learn: 0.3805968	total: 393ms	remaining: 21.4s
18:	learn: 0.3779421	total: 412ms	remaining: 21.3s
19:	learn:

169:	learn: 0.3291964	total: 3.67s	remaining: 17.9s
170:	learn: 0.3290313	total: 3.71s	remaining: 18s
171:	learn: 0.3289290	total: 3.74s	remaining: 18s
172:	learn: 0.3287925	total: 3.77s	remaining: 18s
173:	learn: 0.3285479	total: 3.79s	remaining: 18s
174:	learn: 0.3284196	total: 3.82s	remaining: 18s
175:	learn: 0.3283055	total: 3.87s	remaining: 18.1s
176:	learn: 0.3282299	total: 3.89s	remaining: 18.1s
177:	learn: 0.3281096	total: 3.91s	remaining: 18.1s
178:	learn: 0.3279820	total: 3.94s	remaining: 18s
179:	learn: 0.3279052	total: 3.96s	remaining: 18s
180:	learn: 0.3278013	total: 3.98s	remaining: 18s
181:	learn: 0.3276979	total: 4s	remaining: 18s
182:	learn: 0.3275371	total: 4.02s	remaining: 18s
183:	learn: 0.3274416	total: 4.04s	remaining: 17.9s
184:	learn: 0.3273372	total: 4.07s	remaining: 17.9s
185:	learn: 0.3271951	total: 4.08s	remaining: 17.9s
186:	learn: 0.3271101	total: 4.11s	remaining: 17.9s
187:	learn: 0.3269828	total: 4.13s	remaining: 17.8s
188:	learn: 0.3268779	total: 4.15s	

332:	learn: 0.3128156	total: 7.15s	remaining: 14.3s
333:	learn: 0.3127259	total: 7.17s	remaining: 14.3s
334:	learn: 0.3125841	total: 7.19s	remaining: 14.3s
335:	learn: 0.3124938	total: 7.21s	remaining: 14.2s
336:	learn: 0.3124195	total: 7.23s	remaining: 14.2s
337:	learn: 0.3123433	total: 7.25s	remaining: 14.2s
338:	learn: 0.3122902	total: 7.27s	remaining: 14.2s
339:	learn: 0.3121976	total: 7.29s	remaining: 14.1s
340:	learn: 0.3121296	total: 7.3s	remaining: 14.1s
341:	learn: 0.3120274	total: 7.33s	remaining: 14.1s
342:	learn: 0.3119441	total: 7.35s	remaining: 14.1s
343:	learn: 0.3118807	total: 7.37s	remaining: 14s
344:	learn: 0.3118260	total: 7.39s	remaining: 14s
345:	learn: 0.3117176	total: 7.41s	remaining: 14s
346:	learn: 0.3116236	total: 7.43s	remaining: 14s
347:	learn: 0.3115330	total: 7.45s	remaining: 14s
348:	learn: 0.3114851	total: 7.47s	remaining: 13.9s
349:	learn: 0.3113883	total: 7.49s	remaining: 13.9s
350:	learn: 0.3112942	total: 7.5s	remaining: 13.9s
351:	learn: 0.3112314	to

495:	learn: 0.2995918	total: 10.4s	remaining: 10.6s
496:	learn: 0.2995160	total: 10.4s	remaining: 10.6s
497:	learn: 0.2994438	total: 10.5s	remaining: 10.5s
498:	learn: 0.2993725	total: 10.5s	remaining: 10.5s
499:	learn: 0.2992903	total: 10.5s	remaining: 10.5s
500:	learn: 0.2992316	total: 10.5s	remaining: 10.5s
501:	learn: 0.2991414	total: 10.5s	remaining: 10.4s
502:	learn: 0.2990803	total: 10.6s	remaining: 10.4s
503:	learn: 0.2990129	total: 10.6s	remaining: 10.4s
504:	learn: 0.2989461	total: 10.6s	remaining: 10.4s
505:	learn: 0.2988820	total: 10.6s	remaining: 10.4s
506:	learn: 0.2988194	total: 10.6s	remaining: 10.3s
507:	learn: 0.2987445	total: 10.6s	remaining: 10.3s
508:	learn: 0.2986681	total: 10.7s	remaining: 10.3s
509:	learn: 0.2985692	total: 10.7s	remaining: 10.3s
510:	learn: 0.2984809	total: 10.7s	remaining: 10.2s
511:	learn: 0.2984207	total: 10.7s	remaining: 10.2s
512:	learn: 0.2983763	total: 10.7s	remaining: 10.2s
513:	learn: 0.2982893	total: 10.8s	remaining: 10.2s
514:	learn: 

663:	learn: 0.2876990	total: 13.7s	remaining: 6.92s
664:	learn: 0.2876277	total: 13.7s	remaining: 6.9s
665:	learn: 0.2875516	total: 13.7s	remaining: 6.88s
666:	learn: 0.2874653	total: 13.7s	remaining: 6.86s
667:	learn: 0.2874085	total: 13.8s	remaining: 6.84s
668:	learn: 0.2873366	total: 13.8s	remaining: 6.83s
669:	learn: 0.2872880	total: 13.8s	remaining: 6.8s
670:	learn: 0.2872434	total: 13.8s	remaining: 6.78s
671:	learn: 0.2871641	total: 13.9s	remaining: 6.76s
672:	learn: 0.2870932	total: 13.9s	remaining: 6.74s
673:	learn: 0.2870211	total: 13.9s	remaining: 6.72s
674:	learn: 0.2869583	total: 13.9s	remaining: 6.7s
675:	learn: 0.2868715	total: 13.9s	remaining: 6.68s
676:	learn: 0.2868103	total: 14s	remaining: 6.66s
677:	learn: 0.2867604	total: 14s	remaining: 6.64s
678:	learn: 0.2867009	total: 14s	remaining: 6.62s
679:	learn: 0.2866363	total: 14s	remaining: 6.6s
680:	learn: 0.2865727	total: 14s	remaining: 6.58s
681:	learn: 0.2864885	total: 14.1s	remaining: 6.55s
682:	learn: 0.2864181	tota

830:	learn: 0.2773043	total: 17s	remaining: 3.45s
831:	learn: 0.2772432	total: 17s	remaining: 3.43s
832:	learn: 0.2771874	total: 17s	remaining: 3.41s
833:	learn: 0.2771155	total: 17s	remaining: 3.39s
834:	learn: 0.2770572	total: 17s	remaining: 3.37s
835:	learn: 0.2769967	total: 17.1s	remaining: 3.35s
836:	learn: 0.2769331	total: 17.1s	remaining: 3.33s
837:	learn: 0.2769005	total: 17.1s	remaining: 3.3s
838:	learn: 0.2768264	total: 17.1s	remaining: 3.28s
839:	learn: 0.2767647	total: 17.1s	remaining: 3.26s
840:	learn: 0.2767070	total: 17.1s	remaining: 3.24s
841:	learn: 0.2766493	total: 17.2s	remaining: 3.22s
842:	learn: 0.2765902	total: 17.2s	remaining: 3.2s
843:	learn: 0.2765202	total: 17.2s	remaining: 3.18s
844:	learn: 0.2764676	total: 17.2s	remaining: 3.16s
845:	learn: 0.2763817	total: 17.2s	remaining: 3.14s
846:	learn: 0.2763191	total: 17.3s	remaining: 3.12s
847:	learn: 0.2762648	total: 17.3s	remaining: 3.1s
848:	learn: 0.2761859	total: 17.3s	remaining: 3.08s
849:	learn: 0.2761281	tot

993:	learn: 0.2677754	total: 20.2s	remaining: 122ms
994:	learn: 0.2677176	total: 20.2s	remaining: 102ms
995:	learn: 0.2676826	total: 20.2s	remaining: 81.2ms
996:	learn: 0.2676276	total: 20.2s	remaining: 60.9ms
997:	learn: 0.2675801	total: 20.3s	remaining: 40.6ms
998:	learn: 0.2674946	total: 20.3s	remaining: 20.3ms
999:	learn: 0.2674357	total: 20.3s	remaining: 0us


Pipeline(steps=[('imputer', ArbitraryNumberImputer(arbitrary_number=-9999)),
                ('encoder', MeanEncoder()), ('scaler', MinMaxScaler()),
                ('model',
                 VotingClassifier(estimators=[('m1',
                                               <catboost.core.CatBoostClassifier object at 0x000001C9C8E4CB88>),
                                              ('m2', RandomForestClassifier()),
                                              ('m3',
                                               GradientBoostingClassifier()),
                                              ('m4', LGBMClassifier()),
                                              ('m5',
                                               XGBClassifier(base_...
                                                             interaction_constraints=None,
                                                             learning_rate=None,
                                                             max_bin=None,
      

In [94]:
y_pred = pipeline.predict_proba(features_test)

In [95]:
y_pred = y_pred[:, 1]

In [96]:
len(y_pred), sum(y_pred)

(26824, 22566.91333981701)

In [97]:
submit_data = pd.DataFrame(data=y_pred, columns=['Predicted'])

In [98]:
submit_data.to_csv(f'{PATH_TO_PREPARED_DATA}/submit.csv', index_label='Id', sep=",")

In [143]:
submit_data[submit_data['Predicted']==0]

,Predicted
